In [7]:
#!/usr/bin/python
# -*- coding:utf-8 -*-

import os
import time
import copy
import argparse
import shutil
import torch, cv2
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
import torch.utils.data as Data
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

# from logger import Logger
from preprocessingdata import myImageFloder

import tensorflow as tf
import numpy as np
import scipy.misc


try:
    from StringIO import StringIO      # Python 2.7
except ImportError:
    from io import BytesIO             # Python 3.x

class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)


    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    
    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
        # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format='png')

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                               height=img.shape[0],
                               width=img.shape[1])

            # Create a Summary value
        img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag,i),image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)

        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""
        
        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)
        
        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))
        
        # Drop the start of the first bin
        bin_edges = bin_edges[1:]
        
        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()


# Training settings
parser = argparse.ArgumentParser(description='Pytorch Scene Recognition Training')
parser.add_argument('--batch_size', type=int, default=100, metavar='N',
                   help='input batch size for training(default:100)')
parser.add_argument('--num_classes', type=int, default=80, metavar='N',
                    help='the total classes kinds number of scene classifier')
parser.add_argument('--test_batch_size', type=int, default=100, metavar='N',
                   help='input batch size for testing')
parser.add_argument('--epochs', type=int, default=100, metavar='N',
                   help='number of epochs to train')
parser.add_argument('--start_epoch', type=int, default=0, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--resume', type=str, default='', metavar='PATH',
                    help='path to latest checkpoint (default:none)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
                   help='initial learning rate')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                   help='SGD momentum')
parser.add_argument('--weight_decay', '--wd', type=float, default=1e-4, metavar='W',
                   help='wight decay(default: 1e-4)')
parser.add_argument('--no_cuda', action='store_true', default=False,
                   help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                   help='random seed(default:1)')
parser.add_argument('--log_inteval', type=int, default=50, metavar='N',
                   help='how many batches to wait before logging training status')
parser.add_argument('--arch', '-a', default='alexnet', metavar='ARCH',
                   help='the deep model architecture (default: alexnet)')

parser.add_argument('--train_log', type=str, default='/home/haoyanlong/AI/logs/alexnet_log/train',
                    help='the train parameters logging')
parser.add_argument('--val_log', type=str, default='/home/haoyanlong/AI/logs/alexnet_log/val',
                    help='the validation curracy logging')
parser.add_argument('--checkpoint_path', type=str, default='/home/haoyanlong/AI/modelcheckpoint/alexnet/',
                    help='the directory of model checkpoint saved')
best_prec1 = 0



class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()


    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0


    def update(self, val, n=1):
        # Because the CrossEntropyLoss() is calculated averagely on mini-batch
        # size_average (bool, optional)–By default, the losses are averaged over observations for each minibatch
        # so, val * n means the total loss on the mini-batch
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """
    Sets the learning rate to the initial LR decayed by 10 every 30epochs
    :param optimizer:the optimizer class
    :param epoch:int the training epoch
    :return:the lr
    """
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_groups in optimizer.param_groups:
        param_groups['lr'] = lr


def accuracy(output, label, topk=(1,3,5)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = label.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = torch.eq(pred, label.view(1,-1).expand_as(pred))

    res = []

    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """
    :param state: the model state
    :param is_best: the current whether is the best
    :param filename: the name of filename saved
    :return:
    """
    torch.save(state, filename + '_lastest.pth.tar')
    if is_best:
        shutil.copyfile(filename + '_lastest.pth.tar', filename + '_best.path.tar')


def finemodel(predmodel):
    """
    :param predmodel: the model pretrained on places365
    :return: the model fine changed
    """
    model = torch.load(predmodel)

    for i, params in enumerate(model.parameters()):
        params.requires_grad = False

    # The new classifier removes the last layer
    new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
    # the size of last output features: 4096
    num_features = model.classifier[-1].in_features

    new_classifier.add_module('scene_classify', nn.Linear(num_features, args.num_classes))

    model.classifier = new_classifier
    return model


# fine-tuning the predmodel
def train(train_dataset, predmodel, criterion, optimizer, epoch):
    """
    :param predmodel: (model) fine-changed model
    :param epoch: the current training epoch
    :return:
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top3 = AverageMeter()
    top5 = AverageMeter()

    # logger event
    logger = Logger(args.train_log)

    # switch to train mode
    predmodel.train()

    end = time.time()

    for batch_idx, (image, label) in enumerate(train_dataset,1):

        # measure data loading time
        data_time.update(time.time() - end)

        if not args.no_cuda and torch.cuda.is_available():
            image, label = image.cuda(), label.cuda()

        image, label = Variable(image), Variable(label)

        # compute output
        output = predmodel(image)
        loss = criterion(output, label)

        # measure accuracy and record loss
        prec1, prec3, prec5 = accuracy(output=output.data, label=label, topk=(1,3,5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1, input.size(0))
        top3.update(prec3, input.size(0))
        top5.update(prec5, input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if batch_idx % args.log_interval == 99:
            print ('Train Epoch: [{0}], Train processing:[{1} / {2}\t'
                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                   'Data {data_time.val:.3f} ({data_time.avg: .3f})\t'
                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                   'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                   'Prec@3 {top3.val:.3f} ({top3.avg:.3f})\t'
                   'Prec@5 {top5.val:.3f} ({top5.avg:.3f})\t'.format(
                    epoch, batch_idx, len(train_dataset),
                    batch_time=batch_time,
                    data_time=data_time,
                    loss=losses,
                    top1=top1,top3=top3,top5=top5)
            )

            ################# tensorboard logging ######################
            step = batch_idx * (epoch + 1)
            # (1) Log the scalar values
            info = {
                'loss_val': losses.val,
                'loss_avg': losses.avg,
                'top1_val': top1.val,
                'top1_avg': top1.avg,
                'top3_val': top3.val,
                'top3_avg': top3.avg,
                'top5_val': top5.val,
                'top5_avg': top5.avg
                }
            for tag, value in info.items():
                logger.scalar_summary(tag, value, step)
            # (2) Log values and gradients of the parameter (histogram)
            for tag, value in predmodel.named_parameters():
                tag = tag.replace('.','/')
                logger.histo_summary(tag, value.data.cpu().numpy(), step)
                logger.histo_summary(tag+'/grad', value.grad.cpu().numpy(), step)

            # (3) Log the images
            info = {
                'images': image.view(-1,3,224,224).cpu().numpy()[:10]
                }
            for tag, image in info.items():
                logger.image_summary(tag, image, step)


# Validation the finedmodel
def validate(validation_dataset, predmodel, criterion):

    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top3 = AverageMeter()
    top5 = AverageMeter()

    # logger event
    logger = Logger(args.val_log)

    # switch to evaluate mode
    predmodel.eval()

    end = time.time()

    for idx, (image, label) in enumerate(validation_dataset):
        if not args.no_cuda and torch.cuda.is_available():
            image, label = image.cuda(), label.cuda()

        image, label = Variable(image, volatile=True), Variable(label, volatile=True)

        # compute output
        output = predmodel(image)
        loss = criterion(output, label)

        # measure accuracy and record loss
        prec1, prec3, prec5 = accuracy(output=output.data, label=label, topk=(1, 3, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1, input.size(0))
        top3.update(prec3, input.size(0))
        top5.update(prec5, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if idx % args.log_interval == 0:
            print ('Test: [{0}/{1}]\t'
                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                   'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                   'Prec@3 {top3.val:.3f} ({top3.avg:.3f})\t'
                   'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                    idx,len(validation_dataset),
                    batch_time=batch_time,
                    loss = losses,
                    top1=top1,
                    top3=top3,
                    top5=top5))

            ################# tensorboard logging ######################
            # (1) Log the scalar values
            info = {
                'loss_val': losses.val,
                'loss_avg': losses.avg,
                'top1_val': top1.val,
                'top1_avg': top1.avg,
                'top3_val': top3.val,
                'top3_avg': top3.avg,
                'top5_val': top5.val,
                'top5_avg': top5.avg
            }
            for tag, value in info.items():
                logger.scalar_summary(tag, value, idx)
            # (2) Log values and gradients of the parameter (histogram)
            for tag, value in predmodel.named_parameters():
                tag = tag.replace('.', '/')
                logger.histo_summary(tag, value.data.cpu().numpy(), idx)
                logger.histo_summary(tag + '/grad', value.grad.cpu().numpy(), idx)

            # (3) Log the images
            info = {
                'images': image.view(-1, 3, 224, 224).cpu().numpy()[:10]
            }
            for tag, image in info.items():
                logger.image_summary(tag, image, idx)
    # return top1.avg, top3.avg, top5.avg
    return top1.avg


def main():

    global args, best_prec1
    args = parser.parse_args()
    print args
    
    """
    # Preparing data
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    train_dataloader = myImageFloder(
        root='/home1/haoyanlong/AiChallenger_datasets/scenerecognition/ai_challenger_scene_train_20170904/scene_train_images_20170904',
        label='/home1/haoyanlong/AiChallenger_datasets/scenerecognition/ai_challenger_scene_train_20170904/train_data.txt',
        transform=transforms.Compose(
            [transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize]
        ))
    train_data = Data.DataLoader(dataset=train_dataloader,
                             batch_size=args.batch_size,
                             shuffle=True,
                             num_workers=2)


    val_dataloader = myImageFloder(
        root='/home1/haoyanlong/AiChallenger_datasets/scenerecognition/ai_challenger_scene_validation_20170908/scene_validation_images_20170908',
        label='/home1/haoyanlong/AiChallenger_datasets/scenerecognition/ai_challenger_scene_validation_20170908/validation_data.txt',
        transform=transforms.Compose(
            [transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize]
        ))
    val_data = Data.DataLoader(dataset=val_dataloader,
                               batch_size=args.batch_size,
                               shuffle=False,
                               num_workers=2)


    # create model
    print "=> creating model"
    model = finemodel("/home/haoyanlong/AI/place365_premodel/whole_{}_places365.pth.tar".format(args.arch))
    print model

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print ("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']

            model.load_state_dict(checkpoint['state_dict'])
            print ("=> loaded checkpoint '{}' (epoch {})".format(args.resume, checkpoint['epoch']))
        else:
            print ("=> no checkpoint found at '{}'".format(args.resume))


    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.SGD(model.scene_classify.parameters(),
                          lr=args.lr,
                          momentum=args.momentum,
                          weight_decay=args.weight_decay)


    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer=optimizer,
                             epoch=epoch)

        # train for one epoch
        train(train_datset=train_data, predmodel=model, criterion=criterion, optimizer=optimizer, epoch=epoch)

        # evalute on validation set
        prec1 = validation(validation_dataset=val_data, predmodel=model, criterion=criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)

        save_checkpoint({
            'epoch': epoch + 1,
            'arch': args.arch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1},
            is_best=is_best,
            filename=args.checkpoint_path + args.arch.lower())
    """


if __name__ == '__main__':
    main()


usage: __main__.py [-h] [--batch_size N] [--num_classes N]
                   [--test_batch_size N] [--epochs N] [--start_epoch N]
                   [--resume PATH] [--lr LR] [--momentum M] [--weight_decay W]
                   [--no_cuda] [--seed S] [--log_inteval N] [--arch ARCH]
                   [--train_log TRAIN_LOG] [--val_log VAL_LOG]
                   [--checkpoint_path CHECKPOINT_PATH]
__main__.py: error: unrecognized arguments: -f /run/user/1014/jupyter/kernel-3b10c2ec-1aa3-476d-b169-a9c1eef178f9.json


SystemExit: 2

In [1]:
import argparse

parser = argparse.ArgumentParser(description='Pytorch Scene Recognition Training')
parser.add_argument('--batch_size', type=int, default=100, metavar='N',
                   help='input batch size for training(default:100)')
parser.add_argument('--num_classes', type=int, default=80, metavar='N',
                    help='the total classes kinds number of scene classifier')
parser.add_argument('--test_batch_size', type=int, default=100, metavar='N',
                   help='input batch size for testing')
parser.add_argument('--epochs', type=int, default=100, metavar='N',
                   help='number of epochs to train')
parser.add_argument('--start_epoch', type=int, default=0, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--resume', type=str, default='', metavar='PATH',
                    help='path to latest checkpoint (default:none)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
                   help='initial learning rate')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                   help='SGD momentum')
parser.add_argument('--weight_decay', '--wd', type=float, default=1e-4, metavar='W',
                   help='wight decay(default: 1e-4)')
parser.add_argument('--no_cuda', action='store_true', default=False,
                   help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                   help='random seed(default:1)')
parser.add_argument('--log_inteval', type=int, default=50, metavar='N',
                   help='how many batches to wait before logging training status')
parser.add_argument('--arch', '-a', default='alexnet', metavar='ARCH',
                   help='the deep model architecture (default: alexnet)')

parser.add_argument('--train_log', type=str, default='/home/haoyanlong/AI/logs/alexnet_log/train',
                    help='the train parameters logging')
parser.add_argument('--val_log', type=str, default='/home/haoyanlong/AI/logs/alexnet_log/val',
                    help='the validation curracy logging')
parser.add_argument('--checkpoint_path', type=str, default='/home/haoyanlong/AI/modelcheckpoint/alexnet/',
                    help='the directory of model checkpoint saved')

def main():

    global args, best_prec1
    args = parser.parse_args()
    print args

if __name__ == '__main__':
    main()

usage: __main__.py [-h] [--batch_size N] [--num_classes N]
                   [--test_batch_size N] [--epochs N] [--start_epoch N]
                   [--resume PATH] [--lr LR] [--momentum M] [--weight_decay W]
                   [--no_cuda] [--seed S] [--log_inteval N] [--arch ARCH]
                   [--train_log TRAIN_LOG] [--val_log VAL_LOG]
                   [--checkpoint_path CHECKPOINT_PATH]
__main__.py: error: unrecognized arguments: -f /run/user/1014/jupyter/kernel-f055fe4f-d241-4a00-b68b-b4075a771b19.json


SystemExit: 2

/home/common/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
%tb

SystemExit: 2